In [24]:
import tensorflow as tf
import numpy as np
import random

In [25]:
#Params
kld_reg= 1
adl_reg=1

fdim=16
zdim=16
sigma=1.3
past_length=8
future_length=12
data_scale=170
enc_past_size=(past_length*2,512,256,fdim)
enc_dest_size=(2,8,16,fdim)
enc_latent_size=(2*fdim,8,50,2*zdim)
dec_size=(fdim + zdim,1024,512,1024,2)
predictor_size=(2*fdim,1024,512,256,2*(future_length-1))
learning_rate=0.0003

In [26]:
def loadData(file_path: str):
  npz = np.load(file_path, allow_pickle=True)
  return npz['observations'], npz['obs_speed'], npz['targets'], npz[
      'target_speed'], npz['mean'], npz['std']

In [27]:
class Dense(tf.Module):
  def __init__(self, input_dim, output_size, name=None):
    super(Dense, self).__init__(name=name)
    self.w = tf.Variable(tf.random.uniform([input_dim, output_size],-(1.0/input_dim)**0.5,(1.0/input_dim)**0.5 ),name='w',dtype=tf.float32,trainable=True)
    self.b = tf.Variable(tf.random.uniform([output_size],-(1.0/input_dim)**0.5,(1.0/input_dim)**0.5 ), name='b',dtype=tf.float32,trainable=True)
  def __call__(self, x):
    #x = tf.constant(x,dtype=tf.float32)
    y = tf.matmul(x, self.w) + self.b
    return y

class FullyConnectedNeuralNet(tf.Module):
  def __init__(self,sizes, name=None):
    super(FullyConnectedNeuralNet, self).__init__(name=name)
    self.layers = []
    with self.name_scope:
      for i in range(len(sizes)-1):
        self.layers.append(Dense(input_dim=sizes[i], output_size=sizes[i+1]))
  @tf.Module.with_name_scope
  def __call__(self, x):
    for i,layer in enumerate(self.layers):
        x=layer(x)
        if i != len(self.layers)-1:
            x = tf.nn.relu(x)
    return x

In [28]:
class MainModel(tf.Module):
  def __init__(self,name=None):
    super(MainModel, self).__init__(name=name)

    self.zdim = zdim
    self.sigma = sigma

    self.pastEncoder = FullyConnectedNeuralNet(enc_past_size)

    self.destEncoder = FullyConnectedNeuralNet(enc_dest_size)

    self.latentDistributionEncoder = FullyConnectedNeuralNet(enc_latent_size)

    self.latentDistributionDecoder = FullyConnectedNeuralNet(dec_size)

    self.predictorNetwork = FullyConnectedNeuralNet(predictor_size)

  def forward(self, x, dest = []):

    if len(dest):
        self.training=True
    else:
        self.training=False
        
    # encode
    traj_past_ftr = self.pastEncoder(x)
    #print(f"ftraj max {ftraj.numpy().max()}")
    if not self.training:
        z = tf.random.normal((x.shape[0], self.zdim),0,self.sigma)

    else:
        dest_ftr = self.destEncoder(dest)
        #print(f"dest_features Max {dest_features.numpy().max()}")

        concat_ftr = tf.concat((traj_past_ftr, dest_ftr), axis = 1)
        latent =  self.latentDistributionEncoder(concat_ftr)
        mu = latent[:, 0:self.zdim] # 2-d array
        logvar = latent[:, self.zdim:] # 2-d array

        var = tf.math.exp(logvar*0.5)
        #print(f"var {var}")
        eps = tf.random.normal(var.shape)
        z = eps*var + mu
        #print(f"z -> {z}")


    latentDistributionDecoder_input = tf.concat((traj_past_ftr, z), axis = 1)
    generated_dest = self.latentDistributionDecoder(latentDistributionDecoder_input)
    
    if self.training:
        generated_dest_ftr = self.destEncoder(generated_dest)
        prediction_ftr = tf.concat((traj_past_ftr, generated_dest_ftr), axis = 1)
        pred_future = self.predictorNetwork(prediction_ftr)
        
        return (generated_dest, mu, logvar, pred_future)
    else:
        return generated_dest

  def predict(self, past, generated_dest):
        
    traj_past_ftr = self.pastEncoder(past)
    generated_dest_ftr = self.destEncoder(generated_dest)
    prediction_ftr = tf.concat((traj_past_ftr, generated_dest_ftr), axis = 1)
    future_traj = self.predictorNetwork(prediction_ftr)
    return future_traj

In [29]:
def calculate_loss(dest, dest_rec, mean, log_var, future, future_rec):
    
    rcl = tf.math.reduce_mean(tf.keras.metrics.mean_squared_error(dest, dest_rec))
    adl = tf.math.reduce_mean(tf.keras.metrics.mean_squared_error(future, future_rec))

    kld = -0.5 * tf.math.reduce_sum(1 + log_var - mean**2 - tf.math.exp(log_var))

    return rcl, kld, adl

In [30]:
def next_batch(X,batchSize):
    start = random.randint(0, len(X)-batchSize)
    return X[start:start+batchSize]

In [31]:
def train(trajx,model,optimizer):
    train_loss = 0
    total_rcl, total_kld, total_adl = 0, 0, 0
    
    traj = trajx - trajx[:, :1, :]
    traj *= data_scale

    x = traj[:, :past_length, :]
    y = traj[:, past_length:, :]

    x = x.reshape(-1, x.shape[1]*x.shape[2]) # (x,y,x,y ... )
    dest = y[:, -1, :]
    future = y[:, :-1, :].reshape(y.shape[0],-1)
                
    #x.astype(np.float64)
    print(f"trajx-> {trajx.shape}")

    with tf.GradientTape() as tape:
        x=tf.constant(x,dtype=tf.float32)
        tape.watch(x)
        dest_rec, mu, var, future_rec = model.forward(x, dest=dest)
        #print(f"dest_recon {dest_recon}")
        #print(f"mu {mu}")
        #print(f"var {var}")
        #print(f"interpolated_future {interpolated_future}")
                    
        rcl, kld, adl = calculate_loss(dest, dest_rec, mu, var, future, future_rec)
                    
        loss = rcl + kld * kld_reg + adl * adl_reg
        grad_sub = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grad_sub, model.trainable_variables))
                    
        #print(f"total Loss {loss}")
        #print(f"rcl Loss {rcl}")
        #print(f"kld Loss {kld}")
        #print(f"adl Loss {adl}")
    train_loss+=loss
    total_rcl+=rcl
    total_kld+=kld
    total_adl+=adl
    return train_loss, total_rcl, total_kld, total_adl
                

In [32]:
def test(trajx, model, best_of_n = 1):
    
    traj = trajx - trajx[:, :1, :]
    traj *= data_scale

    x = traj[:, :past_length, :]
    y = traj[:, past_length:, :]

    x = x.reshape(-1, x.shape[1]*x.shape[2])

    dest = y[:, -1, :]
    
    destination_errors = []
    dectination_recs = []
    
    for _ in range(best_of_n):
        x=tf.constant(x,dtype=tf.float32)
        dest_rec = model.forward(x)
        dectination_recs.append(np.array(dest_rec))

        error = np.linalg.norm(dest_rec - dest, axis = 1)
        destination_errors.append(error)

    destination_errors = np.array(destination_errors)
    dectination_recs = np.array(dectination_recs)
    # average error
    avg_dest_error = np.mean(destination_errors)

    indices = np.argmin(destination_errors, axis = 0)

    best_dest = dectination_recs[indices,np.arange(x.shape[0]),  :]

    # taking the minimum error out of all guess
    dest_error = np.mean(np.min(destination_errors, axis = 0))

    future_dest = model.predict(x, best_dest)
    # final overall prediction
    predicted_future = np.concatenate((future_dest, best_dest), axis = 1)
    predicted_future = np.reshape(predicted_future, (-1, future_length, 2))
    # ADE error
    overall_error = np.mean(np.linalg.norm(y - predicted_future, axis = 2))

    overall_error /= data_scale
    dest_error /= data_scale
    avg_dest_error /= data_scale
    #print('Test time error in destination best: {:0.3f} and mean: {:0.3f}'.format(dest_error, avg_dest_error))
    #print('Test time error overall (ADE) best: {:0.3f}'.format(overall_error))

    return overall_error, dest_error, avg_dest_error

            

In [33]:
def run_train():
    observations, _, targets, _, _, _ = loadData('./data/eth/eth_train.npz')
    train_dataset = np.concatenate([observations, targets], axis=1)
    observations, _, targets, _, _, _ = loadData('./data/eth/eth_test.npz')
    test_dataset = np.concatenate([observations, targets], axis=1)
    
    epochs = 8
    batchSize=100
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model=MainModel()
    N=20
    best_test_loss = 50 # start saving after this threshold
    best_endpoint_loss = 50
    for epo in range(epochs):
            #print(f"Epoch : {epo+1}")
            for it in range(int(len(train_dataset)/batchSize)):
                trajx_train = next_batch(train_dataset,batchSize)
                trajx_test = next_batch(test_dataset,len(test_dataset))
                
                train_loss, rcl, kld, adl = train(trajx_train, model,optimizer)
                test_loss, final_point_loss_best, final_point_loss_avg = test(trajx_test, model, best_of_n = N)
                
                if best_test_loss > test_loss:
                    print("Epoch: ", epo+1)
                    print('################## BEST PERFORMANCE {:0.2f} ########'.format(test_loss))
                    best_test_loss = test_loss
                """"
                if best_test_loss < 10.25:
                    save_path = './content/trained.pt'
                """

                if final_point_loss_best < best_endpoint_loss:
                    best_endpoint_loss = final_point_loss_best

                print("Train Loss", train_loss)
                print("RCL", rcl)
                print("KLD", kld)
                print("ADL", adl)
                print("Test ADE", test_loss)
                print("Test Average FDE (Across  all samples)", final_point_loss_avg)
                print("Test Min FDE", final_point_loss_best)
                print("Test Best ADE Loss So Far (N = {})".format(N), best_test_loss)
                print("Test Best Min FDE (N = {})".format(N), best_endpoint_loss)

In [34]:
run_train()

trajx-> (100, 20, 2)
Epoch:  1
################## BEST PERFORMANCE 4.98 ########
Train Loss tf.Tensor(377035.44, shape=(), dtype=float32)
RCL tf.Tensor(163280.64, shape=(), dtype=float32)
KLD tf.Tensor(126231.86, shape=(), dtype=float32)
ADL tf.Tensor(87522.95, shape=(), dtype=float32)
Test ADE 4.980601109728656
Test Average FDE (Across  all samples) 6.494669835707721
Test Min FDE 6.494182272518382
Test Best ADE Loss So Far (N = 20) 4.980601109728656
Test Best Min FDE (N = 20) 6.494182272518382
trajx-> (100, 20, 2)
Epoch:  1
################## BEST PERFORMANCE 4.98 ########
Train Loss tf.Tensor(104233656.0, shape=(), dtype=float32)
RCL tf.Tensor(885309.75, shape=(), dtype=float32)
KLD tf.Tensor(102859240.0, shape=(), dtype=float32)
ADL tf.Tensor(489100.3, shape=(), dtype=float32)
Test ADE 4.976920542845022
Test Average FDE (Across  all samples) 6.482076487821691
Test Min FDE 6.481532197840074
Test Best ADE Loss So Far (N = 20) 4.976920542845022
Test Best Min FDE (N = 20) 6.481532197840

Epoch:  1
################## BEST PERFORMANCE 4.84 ########
Train Loss tf.Tensor(555146.6, shape=(), dtype=float32)
RCL tf.Tensor(368520.6, shape=(), dtype=float32)
KLD tf.Tensor(1265.4762, shape=(), dtype=float32)
ADL tf.Tensor(185360.56, shape=(), dtype=float32)
Test ADE 4.843429917444865
Test Average FDE (Across  all samples) 6.069297162224265
Test Min FDE 6.067764102711397
Test Best ADE Loss So Far (N = 20) 4.843429917444865
Test Best Min FDE (N = 20) 6.067764102711397
trajx-> (100, 20, 2)
Epoch:  1
################## BEST PERFORMANCE 4.83 ########
Train Loss tf.Tensor(124156.94, shape=(), dtype=float32)
RCL tf.Tensor(81866.59, shape=(), dtype=float32)
KLD tf.Tensor(174.85684, shape=(), dtype=float32)
ADL tf.Tensor(42115.49, shape=(), dtype=float32)
Test ADE 4.827496035453462
Test Average FDE (Across  all samples) 6.025880342371323
Test Min FDE 6.024334357766544
Test Best ADE Loss So Far (N = 20) 4.827496035453462
Test Best Min FDE (N = 20) 6.024334357766544
trajx-> (100, 20, 2)
Ep

Epoch:  1
################## BEST PERFORMANCE 3.75 ########
Train Loss tf.Tensor(611954.6, shape=(), dtype=float32)
RCL tf.Tensor(379526.84, shape=(), dtype=float32)
KLD tf.Tensor(1912.0908, shape=(), dtype=float32)
ADL tf.Tensor(230515.72, shape=(), dtype=float32)
Test ADE 3.745256428181297
Test Average FDE (Across  all samples) 3.617937514361213
Test Min FDE 3.61180419921875
Test Best ADE Loss So Far (N = 20) 3.745256428181297
Test Best Min FDE (N = 20) 3.61180419921875
trajx-> (100, 20, 2)
Epoch:  1
################## BEST PERFORMANCE 3.61 ########
Train Loss tf.Tensor(698613.9, shape=(), dtype=float32)
RCL tf.Tensor(461875.7, shape=(), dtype=float32)
KLD tf.Tensor(977.603, shape=(), dtype=float32)
ADL tf.Tensor(235760.56, shape=(), dtype=float32)
Test ADE 3.6068830568652146
Test Average FDE (Across  all samples) 3.5885752958409927
Test Min FDE 3.5821734260110296
Test Best ADE Loss So Far (N = 20) 3.6068830568652146
Test Best Min FDE (N = 20) 3.5821734260110296
trajx-> (100, 20, 2)


Train Loss tf.Tensor(5057.8965, shape=(), dtype=float32)
RCL tf.Tensor(1314.1207, shape=(), dtype=float32)
KLD tf.Tensor(5.848172, shape=(), dtype=float32)
ADL tf.Tensor(3737.9277, shape=(), dtype=float32)
Test ADE 2.8731126159685667
Test Average FDE (Across  all samples) 4.129754638671875
Test Min FDE 4.124413703469669
Test Best ADE Loss So Far (N = 20) 2.7076327969847633
Test Best Min FDE (N = 20) 3.5711504767922793
trajx-> (100, 20, 2)
Train Loss tf.Tensor(198279.66, shape=(), dtype=float32)
RCL tf.Tensor(125650.86, shape=(), dtype=float32)
KLD tf.Tensor(146.71051, shape=(), dtype=float32)
ADL tf.Tensor(72482.086, shape=(), dtype=float32)
Test ADE 2.9339157238781652
Test Average FDE (Across  all samples) 4.311714441636029
Test Min FDE 4.306585334329045
Test Best ADE Loss So Far (N = 20) 2.7076327969847633
Test Best Min FDE (N = 20) 3.5711504767922793
trajx-> (100, 20, 2)
Train Loss tf.Tensor(155203.52, shape=(), dtype=float32)
RCL tf.Tensor(103322.21, shape=(), dtype=float32)
KLD tf

Train Loss tf.Tensor(354355.56, shape=(), dtype=float32)
RCL tf.Tensor(250853.3, shape=(), dtype=float32)
KLD tf.Tensor(165.32828, shape=(), dtype=float32)
ADL tf.Tensor(103336.94, shape=(), dtype=float32)
Test ADE 2.5631550128995633
Test Average FDE (Across  all samples) 3.6866871553308824
Test Min FDE 3.678966926125919
Test Best ADE Loss So Far (N = 20) 2.509396732713773
Test Best Min FDE (N = 20) 3.5676926556755513
trajx-> (100, 20, 2)
Train Loss tf.Tensor(129740.19, shape=(), dtype=float32)
RCL tf.Tensor(99432.47, shape=(), dtype=float32)
KLD tf.Tensor(302.02463, shape=(), dtype=float32)
ADL tf.Tensor(30005.695, shape=(), dtype=float32)
Test ADE 2.6382506092348628
Test Average FDE (Across  all samples) 3.5865776510799634
Test Min FDE 3.5789956485523895
Test Best ADE Loss So Far (N = 20) 2.509396732713773
Test Best Min FDE (N = 20) 3.5676926556755513
trajx-> (100, 20, 2)
Train Loss tf.Tensor(202318.4, shape=(), dtype=float32)
RCL tf.Tensor(134429.84, shape=(), dtype=float32)
KLD tf.

...

Train Loss tf.Tensor(461195.3, shape=(), dtype=float32)
RCL tf.Tensor(332399.03, shape=(), dtype=float32)
KLD tf.Tensor(1206.224, shape=(), dtype=float32)
ADL tf.Tensor(127590.08, shape=(), dtype=float32)
Test ADE 2.267584589551158
Test Average FDE (Across  all samples) 3.6567059685202206
Test Min FDE 3.408121266084559
Test Best ADE Loss So Far (N = 20) 2.1572439880235614
Test Best Min FDE (N = 20) 3.1792832318474264
trajx-> (100, 20, 2)
Train Loss tf.Tensor(856480.7, shape=(), dtype=float32)
RCL tf.Tensor(567331.25, shape=(), dtype=float32)
KLD tf.Tensor(297.80814, shape=(), dtype=float32)
ADL tf.Tensor(288851.62, shape=(), dtype=float32)
Test ADE 2.2825857291478715
Test Average FDE (Across  all samples) 3.6860286937040443
Test Min FDE 3.438586784811581
Test Best ADE Loss So Far (N = 20) 2.1572439880235614
Test Best Min FDE (N = 20) 3.1792832318474264
trajx-> (100, 20, 2)
Train Loss tf.Tensor(158234.33, shape=(), dtype=float32)
RCL tf.Tensor(106410.22, shape=(), dtype=float32)
KLD tf.

Train Loss tf.Tensor(385428.44, shape=(), dtype=float32)
RCL tf.Tensor(264744.2, shape=(), dtype=float32)
KLD tf.Tensor(941.8737, shape=(), dtype=float32)
ADL tf.Tensor(119742.38, shape=(), dtype=float32)
Test ADE 2.1917478636103263
Test Average FDE (Across  all samples) 3.461657355813419
Test Min FDE 3.1913656795726104
Test Best ADE Loss So Far (N = 20) 2.1572439880235614
Test Best Min FDE (N = 20) 3.1792832318474264
trajx-> (100, 20, 2)
Train Loss tf.Tensor(74317.05, shape=(), dtype=float32)
RCL tf.Tensor(42696.19, shape=(), dtype=float32)
KLD tf.Tensor(2519.2476, shape=(), dtype=float32)
ADL tf.Tensor(29101.605, shape=(), dtype=float32)
Test ADE 2.223874867844473
Test Average FDE (Across  all samples) 3.447786937040441
Test Min FDE 3.1727628820082723
Test Best ADE Loss So Far (N = 20) 2.1572439880235614
Test Best Min FDE (N = 20) 3.1727628820082723
trajx-> (100, 20, 2)
Train Loss tf.Tensor(521405.1, shape=(), dtype=float32)
RCL tf.Tensor(339815.28, shape=(), dtype=float32)
KLD tf.Te

Train Loss tf.Tensor(746197.2, shape=(), dtype=float32)
RCL tf.Tensor(489840.3, shape=(), dtype=float32)
KLD tf.Tensor(749.53894, shape=(), dtype=float32)
ADL tf.Tensor(255607.34, shape=(), dtype=float32)
Test ADE 2.1815770280237947
Test Average FDE (Across  all samples) 3.4570603314568014
Test Min FDE 3.190432918772978
Test Best ADE Loss So Far (N = 20) 2.1572439880235614
Test Best Min FDE (N = 20) 3.1727628820082723
trajx-> (100, 20, 2)
Train Loss tf.Tensor(404874.2, shape=(), dtype=float32)
RCL tf.Tensor(322867.4, shape=(), dtype=float32)
KLD tf.Tensor(609.2739, shape=(), dtype=float32)
ADL tf.Tensor(81397.51, shape=(), dtype=float32)
Test ADE 2.175889396688633
Test Average FDE (Across  all samples) 3.461551441865809
Test Min FDE 3.2017675063189337
Test Best ADE Loss So Far (N = 20) 2.1572439880235614
Test Best Min FDE (N = 20) 3.1727628820082723
trajx-> (100, 20, 2)
Train Loss tf.Tensor(655831.3, shape=(), dtype=float32)
RCL tf.Tensor(440323.7, shape=(), dtype=float32)
KLD tf.Tenso

Train Loss tf.Tensor(89492.94, shape=(), dtype=float32)
RCL tf.Tensor(60963.51, shape=(), dtype=float32)
KLD tf.Tensor(1394.2511, shape=(), dtype=float32)
ADL tf.Tensor(27135.172, shape=(), dtype=float32)
Test ADE 2.264963328170447
Test Average FDE (Across  all samples) 3.5813526826746322
Test Min FDE 3.3362455480238973
Test Best ADE Loss So Far (N = 20) 2.1572439880235614
Test Best Min FDE (N = 20) 3.1727628820082723
trajx-> (100, 20, 2)
Train Loss tf.Tensor(470744.75, shape=(), dtype=float32)
RCL tf.Tensor(356794.9, shape=(), dtype=float32)
KLD tf.Tensor(2210.5286, shape=(), dtype=float32)
ADL tf.Tensor(111739.33, shape=(), dtype=float32)
Test ADE 2.3037424034935152
Test Average FDE (Across  all samples) 3.6088001924402575
Test Min FDE 3.362258731617647
Test Best ADE Loss So Far (N = 20) 2.1572439880235614
Test Best Min FDE (N = 20) 3.1727628820082723
trajx-> (100, 20, 2)
Train Loss tf.Tensor(337621.56, shape=(), dtype=float32)
RCL tf.Tensor(225030.2, shape=(), dtype=float32)
KLD tf.

Train Loss tf.Tensor(461608.6, shape=(), dtype=float32)
RCL tf.Tensor(310757.5, shape=(), dtype=float32)
KLD tf.Tensor(729.495, shape=(), dtype=float32)
ADL tf.Tensor(150121.6, shape=(), dtype=float32)
Test ADE 2.262542428747091
Test Average FDE (Across  all samples) 3.6593258128446693
Test Min FDE 3.412829230813419
Test Best ADE Loss So Far (N = 20) 2.1572439880235614
Test Best Min FDE (N = 20) 3.1727628820082723
trajx-> (100, 20, 2)
Train Loss tf.Tensor(8136.2207, shape=(), dtype=float32)
RCL tf.Tensor(3927.0188, shape=(), dtype=float32)
KLD tf.Tensor(1435.2064, shape=(), dtype=float32)
ADL tf.Tensor(2773.9956, shape=(), dtype=float32)
Test ADE 2.2513675208128423
Test Average FDE (Across  all samples) 3.6401593376608457
Test Min FDE 3.3975453096277572
Test Best ADE Loss So Far (N = 20) 2.1572439880235614
Test Best Min FDE (N = 20) 3.1727628820082723
